# 0. Import Libraries


In [1]:
import pandas as pd
import pyodbc
import warnings
from dotenv import load_dotenv
import os

load_dotenv()
password = os.getenv("PASSWORD")

warnings.filterwarnings("ignore")
conn_str = (
    "Driver={ODBC Driver 17 for SQL Server};"
    "Server=localhost,1433;"
    "Database=VSTAY;"
    "UID=sa;"
    f"PWD={password};"
    "TrustServerCertificate=yes;"
)
cnxn = pyodbc.connect(conn_str)
cursor = cnxn.cursor()

# 1. VSTAY

- The database is self-generated, has 16 tables, data is created using Mockaroo
- VSTAY acts as an intermediary application, connecting home owners and people who want to rent out types of accommodation such as: Villa, apartment, bungalow, homestay...


# 2. Which was the most popular Accommodation type (highest number of bookings that are not cancelled)?


In [2]:
query = """
            SELECT at.AccommodationType, COUNT(b.BookingID) AS nbookings
            FROM ACCOMMODATION_TYPE AS at
            JOIN ACCOMMODATION AS a
            ON at.AccommodationTypeID = a.AccommodationTypeID
            JOIN BOOKING AS b
            ON a.AccommodationID = b.AccommodationID
            WHERE b.BookingStatusID = 01
            GROUP BY at.AccommodationType
            ORDER BY nbookings DESC
        """
df = pd.read_sql(query, cnxn)
df

,AccommodationType,nbookings
0,Duplex,44
1,Condotel,43
2,Homestay,40
3,Villa,40
4,Penthouse,36
5,Farmstay,34
6,Resort,33
7,Studio,32
8,Bungalow,26
9,Treehouse,21


# 3. Which accommodation is highly-rated?


In [3]:
query = """
            SELECT a.AccommodationID, a.OwnerIDCardNumber as 'OwnerID', a.PricePerNight, ROUND(AVG(CAST(f.Rating AS FLOAT)), 2) as  'AverageRating'
            FROM ACCOMMODATION a
            JOIN FEEDBACK f
            ON a.AccommodationID = f.AccommodationID
            GROUP BY a.AccommodationID, a.OwnerIDCardNumber, a.PricePerNight
            HAVING ROUND(AVG(CAST(f.Rating AS FLOAT)), 2) > 4 
        """
df = pd.read_sql(query, cnxn)
df

,AccommodationID,OwnerID,PricePerNight,AverageRating
0,ACM000000003,180126128092,1236240.0,4.33
1,ACM000000009,408276384428,4821253.0,4.67
2,ACM000000020,332811653345,1253847.0,4.50
3,ACM000000022,043606517149,4250251.0,4.33
4,ACM000000031,497211879982,2037361.0,5.00
...,...,...,...,...
87,ACM000000934,692395547940,1727031.0,5.00
88,ACM000000968,493958041053,1756983.0,5.00
89,ACM000000971,601289928207,3699514.0,5.00
90,ACM000000982,728812628777,1003361.0,4.50


# 4. Find accommodations with more than 1 room and price per night between 1 million and 2 million VND. Show the amenities and facilities included in each accomodation.


In [4]:
query = """
            SELECT a.AccommodationID, a.AccommodationTypeID, r.AmenityName, f.FacilityName
            FROM ACCOMMODATION as a
            JOIN ACCOMMODATION_TYPE as at
            ON a.AccommodationTypeID = at.AccommodationTypeID
            JOIN AMENITIES_INCLUDED as ai
            ON ai.AccommodationID = a.AccommodationID
            JOIN ROOM_AMENITIES as r
            ON ai.AmenityID = r.AmenityID
            JOIN FACILITIES_INCLUDED as fi
            ON a.AccommodationID = fi.AccommodationID
            JOIN FACILITIES as f
            ON fi.FacilityID = f.FacilityID
            WHERE a.NumberOfRooms>1 and a.PricePerNight between 1000000 and 2000000
        """
df = pd.read_sql(query, cnxn)
df

,AccommodationID,AccommodationTypeID,AmenityName,FacilityName
0,ACM000000013,DP,Irons,Facilities for disabled guests
1,ACM000000013,DP,Refrigerator,Facilities for disabled guests
2,ACM000000020,VL,Refrigerator,Golf Course
3,ACM000000048,VL,TV,Swimming Pool
4,ACM000000097,FS,Pets allowed in room,Car park
...,...,...,...,...
68,ACM000000968,VL,Irons,Night Club
69,ACM000000968,VL,Internet Access,Night Club
70,ACM000000968,VL,Kitchen,Night Club
71,ACM000000979,TH,Washing Machine,Sauna/Spa


# 5. A customer cancelling within 2 days prior to checkin time will not receive a refund. Which customer did not receive a refund for cancelling a booking?


In [4]:
query = """
                SELECT CONCAT(firstname, lastname) AS Name, ga.GuestIDCardNumber AS GuestID, 
                BookingID, a.AccommodationID, AccommodationName, 
                CONVERT(int, datediff(day, DateTimeCancel, CheckInTime)) AS DaysCancelPrior
                FROM BOOKING bk
                JOIN GUEST_ACCOUNT ga
                ON bk.GuestIDCardNumber =ga.GuestIDCardNumber 
                JOIN BOOKING_STATUS bkst
                ON bk.BookingStatusID = bkst.BookingStatusID
                JOIN ACCOMMODATION a
                ON bk.AccommodationId = a.AccommodationID 
                WHERE bk.BookingStatusID = 00 and convert(int, datediff(day, DateTimeCancel, CheckInTime)) <= 2 
                ORDER BY 3 desc 
        """
df = pd.read_sql(query, cnxn)
df

,Name,GuestID,BookingID,AccommodationID,AccommodationName,DaysCancelPrior
0,GinevraCasella,175612879164,BKG000001000,ACM000000272,None,1
1,HildegaardFoley,587224984547,BKG000000998,ACM000000609,"NextEra Energy, Inc.",0
2,CarlottaNesby,616752495092,BKG000000995,ACM000000121,None,0
3,ChlorisBryceson,223562976913,BKG000000976,ACM000000472,"Twenty-First Century Fox, Inc.",1
4,MorganaRyce,613763235417,BKG000000905,ACM000000165,None,0
5,BetsyHadlow,235676639768,BKG000000704,ACM000000259,None,2
6,FreidaFruin,010034327046,BKG000000662,ACM000000835,None,1
7,IsaSevery,523589519925,BKG000000654,ACM000000515,"Plug Power, Inc.",0
8,CassaundraCockerill,404188726807,BKG000000630,ACM000000541,"Sequential Brands Group, Inc.",0
9,MontgomeryGisburne,885961224009,BKG000000543,ACM000000917,"Mettler-Toledo International, Inc.",1


# 6. Find the average price per night and number of accommodations by accommodation type in "Binh Duong" province.


In [6]:
query = """
            SELECT at.AccommodationType, 
            ROUND(AVG(PricePerNight),1) AS AvgPricePerNight, 
            COUNT(a.accommodationid) AS nAccommodations,
            ProvinceName
            FROM ACCOMMODATION a
            JOIN ACCOMMODATION_TYPE at 
            On a.AccommodationTypeID = at.AccommodationTypeID 
            JOIN [CITY/DISTRICT] c
            On a.[City/DistrictID] = c.[City/DistrictID] 
            JOIN PROVINCE p
            On c.ProvinceID = p.ProvinceID 
            WHERE p.ProvinceName = N'Bình Dương'
            GROUP BY at.AccommodationType, ProvinceName
        """
df = pd.read_sql(query, cnxn)
df

,AccommodationType,AvgPricePerNight,nAccommodations,ProvinceName
0,Bungalow,830679.0,1,Bình Dương
1,Condotel,2390969.5,4,Bình Dương
2,Homestay,1978752.0,1,Bình Dương
3,Resort,243286.0,1,Bình Dương
4,Studio,657588.0,1,Bình Dương
5,Treehouse,782447.0,1,Bình Dương
6,Villa,3149814.5,2,Bình Dương


# 7. Execute procedure to find Province and City of the input AccommodationID.


In [ ]:
query = """
            exec location @accommodationid =  'ACM000000001'
        """
df = pd.read_sql(query, cnxn)
df

# 8. Execute procedure to find accommodations with the iput price range


In [6]:
query = """
            exec pricesearch @lower = 300000,  @upper=400000
        """
df = pd.read_sql(query, cnxn)
df

,AccommodationID,AccommodationName,OwnerIDCardNumber,City/DistrictID,StreetAddress,AccommodationTypeID,NumberOfRooms,Capacity,PricePerNight
0,ACM000000167,Charter Financial Corp.,607293056035,120,88 Northfield Center,RS,4,7,346318.0
1,ACM000000220,MFS Intermediate Income Trust,201863600586,126,9395 Hudson Drive,PH,1,13,349648.0
2,ACM000000230,None,011136019710,695,590 Tony Trail,PH,1,18,390938.0
3,ACM000000331,None,434916579525,055,11 Crescent Oaks Center,RS,2,13,306814.0
4,ACM000000462,None,692828980144,156,18 Cottonwood Street,VL,2,13,315714.0
5,ACM000000505,None,179093574641,469,999 Debs Place,DP,4,7,385403.0
6,ACM000000684,"Global Water Resources, Inc.",996224821500,504,91 Kingsford Hill,RS,5,14,363066.0
7,ACM000000731,Nuveen Minnesota Quality Municipal Income Fund,467275938468,332,75514 Pearson Lane,PH,1,13,377081.0
8,ACM000000745,None,467777682461,657,92042 Anderson Drive,CD,1,13,374185.0
9,ACM000000759,None,617216598468,531,3 Golf View Crossing,VL,2,5,357740.0


# 9. Execute procedure to find customers who has bookings within a date range


In [7]:
query = """
            exec customer_list @lower='2021-09-19 15:09:55.000', @upper='2022-03-08 02:25:47.000'
        """
df = pd.read_sql(query, cnxn)
df

,GuestIDCardNumber,FirstName,LastName,GuestIDCardNumber,PhoneNumber,AccommodationName,AccommodationType,OWNER NAME,StreetAddress,PricePerNight,CheckInTime
0,899424741667,Ellynn,Odd,899424741667,3594268745,None,Farmstay,SimonssonAnnamarie,798 Debs Alley,4380479.0,2022-02-16 12:40:14
1,984182264738,Fee,Nolte,984182264738,8036099626,First Majestic Silver Corp.,Penthouse,BoughFaina,7163 Arrowood Center,1236240.0,2021-10-19 00:22:39
2,630455277331,Karrah,Vannini,630455277331,9795601670,"Aptose Biosciences, Inc.",Condotel,HallgarthSimona,14 Autumn Leaf Center,1360795.0,2021-12-11 18:00:58
3,661752637108,Shanda,Lamas,661752637108,8058291701,None,Studio,WeylandDarrick,6 Bellgrove Court,1500200.0,2022-01-03 00:01:45
4,071333138354,Elijah,Parkman,071333138354,3147681492,None,Homestay,DrengOrrin,85833 Eastwood Circle,2263045.0,2021-10-05 04:40:34
...,...,...,...,...,...,...,...,...,...,...,...
103,543728723540,Caspar,Heather,543728723540,5028716517,None,Farmstay,JevonOphelia,823 Walton Crossing,2153587.0,2022-02-16 04:32:06
104,187206351749,Connor,Bes,187206351749,2226835588,None,Homestay,JandacAidan,104 Springs Place,4606364.0,2022-01-12 14:25:28
105,381128111610,Hermine,Dome,381128111610,4329723797,"Urban One, Inc.",Farmstay,BailyWestley,028 School Crossing,4853545.0,2021-11-09 09:15:41
106,961729482812,Diandra,Keunemann,961729482812,9703124742,None,Villa,MaggsGuido,40 Pearson Crossing,3831023.0,2021-09-29 10:51:45


# 10. Execute procedure to increase or decrease the price of an accommdation with the input amount


In [18]:
# Before
query = """
            SELECT *
            FROM Accommodation
            WHERE AccommodationID = 'ACM000000001'
        """
df = pd.read_sql(query, cnxn)
df

,AccommodationID,AccommodationName,OwnerIDCardNumber,City/DistrictID,StreetAddress,AccommodationTypeID,NumberOfRooms,Capacity,PricePerNight
0,ACM000000001,None,844337223887,172,798 Debs Alley,FS,4,19,4380479.0


In [20]:
# After
query = """
            exec price_change @accommodationid='ACM000000001', @change=500000
        """
df = pd.read_sql(query, cnxn)
df

,AccommodationID,AccommodationName,OwnerIDCardNumber,City/DistrictID,StreetAddress,AccommodationTypeID,NumberOfRooms,Capacity,PricePerNight
0,ACM000000001,None,844337223887,172,798 Debs Alley,FS,4,19,4880479.0
